In [3]:
print('Program started')
import vrep
import numpy as np
import cv2
import cv2.aruco as aruco
import sys, time, math
from platform import python_version

import time

##############Comunicação com V-REP##########################
serverIP = '127.0.0.1';
serverPort = 19999; #Esta porta do servidor está sempre aberta
vrep.simxFinish(-1);
clientID=vrep.simxStart(serverIP,serverPort,True,True,5000,5);
#############################################################

windowName = "Imagem-Processada" #Janela de visualização do Vídeo
cv2.namedWindow(windowName, cv2.WINDOW_NORMAL)#Criando nome e tipo da janela
cv2.setWindowProperty(windowName, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

#-- Define Tag
id_to_find = 72
merer_size = 100 #-cm

#-- Get the camera calibration
calib_path = ''
camera_matrix = np.loadtxt(calib_path+'cameraMatrix.txt', delimiter = ',')
#camera_distortion = np.loadtxt(calib_path+'cameraDistortion.txt', delimiter = ',')

aruco_dict = aruco.Dictionary_get(aruco.DICT_6X6_250)
parameters =  aruco.DetectorParameters_create()
#print(parameters)

if clientID!=-1:
    print ('Connected to remote API server')
    print ('Versão OpenCV: ',cv2.__version__)
    print ('Versão Python: ',python_version())
    
    err,visionHandle = vrep.simxGetObjectHandle(clientID,'Vision_sensor',vrep.simx_opmode_oneshot_wait)
    
    while err == 0:
        
        err,visionHandle = vrep.simxGetObjectHandle(clientID,'Vision_sensor',vrep.simx_opmode_oneshot_wait)
        err,res,imgList = vrep.simxGetVisionSensorImage(clientID,visionHandle,0,vrep.simx_opmode_oneshot_wait) #Imagens do sensor
        
        img = (np.array(imgList))[::-1]#Passa de lista para uma array e inverte a array para plotar corretamente
        imgRGB = cv2.flip(np.uint8(img.reshape(720,1280,3)),1) #Transforma em uma imagem RGB e espelha
        
        #gray = cv2.cvtColor(imgRGB, cv2.COLOR_BGR2GRAY) #Passa de RGB para escala de cinza
        #cv2.imshow(windowName,gray) #Mostra a Imagem em escala de cinza
        
        #lists of ids and the corners beloning to each id
        corners, ids, rejectedImgPoints = aruco.detectMarkers(image=imgRGB, dictionary=aruco_dict, parameters=parameters)
        
        #print(corners)
        imgRGB = aruco.drawDetectedMarkers(imgRGB, corners)
        
        #print(rejectedImgPoints)
        #imgRGB = aruco.drawDetectedMarkers(imgRGB, rejectedImgPoints)
        
        #Display the resulting frame
        cv2.imshow(windowName,imgRGB)
        
        if cv2.waitKey(1) == 27: # exit on ESC
            break

    # Now close the connection to V-REP:
    vrep.simxFinish(clientID)

else:
    print ('Failed connecting to remote API server')
print ('Program ended')

cv2.destroyAllWindows()

Program started
Connected to remote API server
Versão OpenCV:  3.4.2
Versão Python:  3.6.5
Program ended
